In [1]:
# !pip install --force-reinstall numpy
# !pip install -U "pandas>=2.2.3"

In [1]:
# libs
import pandas as pd
import numpy as np

In [2]:
# https://www.kaggle.com/datasets/computingvictor/transactions-fraud-datasets?resource=download&select=transactions_data.csv
example_data = pd.read_csv(r"C:\Users\demenev\pet-projects\clients-calculator\data\transactions_data.csv")

In [4]:
example_data['date'] = pd.to_datetime(example_data['date'])
example_data['date'] = example_data['date'].dt.to_period('d')

example_data = example_data[['date', 'client_id']].drop_duplicates()

In [6]:
example_data.to_csv('transactions_data_part.csv')

In [7]:
{1,2,3} - {1,2}

{3}

In [ ]:
example_data[['client_id', 'date']].head()

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
0,7475327,2010-01-01 00:01:00,1556,2972,$-77.00,Swipe Transaction,59935,Beulah,ND,58523.0,5499,NaN
1,7475328,2010-01-01 00:02:00,561,4575,$14.57,Swipe Transaction,67570,Bettendorf,IA,52722.0,5311,NaN
2,7475329,2010-01-01 00:02:00,1129,102,$80.00,Swipe Transaction,27092,Vista,CA,92084.0,4829,NaN
3,7475331,2010-01-01 00:05:00,430,2860,$200.00,Swipe Transaction,27092,Crown Point,IN,46307.0,4829,NaN
4,7475332,2010-01-01 00:06:00,848,3915,$46.41,Swipe Transaction,13051,Harwood,MD,20776.0,5813,NaN


In [16]:
def clients_intersection(clients_prev, clients):
    return len(clients_prev & clients)

clients_intersection_vectorized = np.vectorize(clients_intersection)


In [18]:
example_data_grouped.isna().sum()

year_month       0
clients          0
clients_count    0
clients_prev     1
dtype: int64

In [30]:
example_data['date'] = pd.to_datetime(example_data['date'])
example_data['year_month'] = example_data['date'].dt.to_period('M')

periods = pd.date_range(start=example_data['date'].min(), end=example_data['date'].max(), freq='M').to_period('M')
periods = periods.to_frame(index=False)
periods.columns = ['year_month']

C:\Users\demenev\AppData\Local\Temp\ipykernel_73520\3687602930.py:4: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  periods = pd.date_range(start=example_data['date'].min(), end=example_data['date'].max(), freq='M').to_period('M')


In [40]:


example_data_grouped = example_data.groupby('year_month', as_index=False).agg(clients = ('client_id', set)).sort_values('year_month', ascending=True)
example_data_grouped = periods.merge(example_data_grouped, on='year_month', how='left')
example_data_grouped['clients_count'] = example_data_grouped['clients'].apply(len)
example_data_grouped['clients_prev'] = example_data_grouped['clients'].shift(1, fill_value=set())
example_data_grouped['clients_prev'] = example_data_grouped['clients_prev'].bfill()
example_data_grouped['clients_prev_year'] = example_data_grouped['clients'].shift(12, fill_value=set())
example_data_grouped['clients_prev_year'] = example_data_grouped['clients_prev_year'].bfill()

example_data_grouped['clients_intersection'] = clients_intersection_vectorized(example_data_grouped['clients_prev'], example_data_grouped['clients'])
example_data_grouped['retention_month'] = example_data_grouped['clients_intersection'] / example_data_grouped['clients_prev'].apply(len)
example_data_grouped['churn_month'] = 1 - example_data_grouped['retention_month']

example_data_grouped['clients_intersection_year'] = clients_intersection_vectorized(example_data_grouped['clients_prev_year'], example_data_grouped['clients'])
example_data_grouped['retention_year'] = example_data_grouped['clients_intersection_year'] / example_data_grouped['clients_prev_year'].apply(len)
example_data_grouped['churn_year'] = 1 - example_data_grouped['retention_year']

In [35]:
example_data_grouped.isna().sum()

year_month              0
clients                 0
clients_count           0
clients_prev            0
clients_intersection    0
retention_month         1
dtype: int64

In [47]:
example_data_grouped['year_month'].min().strftime("%Y-%m")

'2010-01'